<a href="https://colab.research.google.com/github/snpushpi/Differential-Privacy-in-Split-Learning/blob/main/SplitNN_Cifar10_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
        
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

# test()

In [27]:
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


In [28]:
num_classes = len(classes)

In [29]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  print(f'GPU: {torch.cuda.current_device()}, {torch.cuda.device_count()}, {torch.cuda.get_device_name(0)}, {torch.cuda.is_available()}')
else: print(f'Device: cpu')


GPU: 0, 1, Tesla T4, True


In [30]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

from torch.utils.data.sampler import SubsetRandomSampler

total_size = len(trainset)

split1 = total_size // 4
split2 = split1*2
split3 = split1*3

indices = list(range(total_size))

alice_idx = indices[:split1]
bob_idx = indices[split1:split2]
mike_idx = indices[split2:split3]
rose_idc = indices[split3:]

alice_sampler = SubsetRandomSampler(alice_idx)
bob_sampler = SubsetRandomSampler(bob_idx)
mike_sampler = SubsetRandomSampler(mike_idx)
rose_sampler = SubsetRandomSampler(rose_idc)


alice_loader = torch.utils.data.DataLoader(trainset, batch_size=128, sampler=alice_sampler)
bob_loader = torch.utils.data.DataLoader(trainset, batch_size=128, sampler=bob_sampler)
mike_loader = torch.utils.data.DataLoader(trainset, batch_size=128, sampler=mike_sampler)
rose_loader = torch.utils.data.DataLoader(trainset, batch_size=128, sampler=rose_sampler)

data_loaders = [alice_loader, bob_loader, mike_loader, rose_loader ]

print(f'Data at alice: {len(alice_sampler)} \t Batches: {len(alice_loader)}')
print(f'Data at bob: {len(bob_sampler)} \t Batches: {len(alice_loader)}')
print(f'Data at mike: {len(mike_sampler)} \t Batches: {len(mike_loader)}')
print(f'Data at rose: {len(rose_sampler)} \t Batches: {len(rose_loader)}')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Data at alice: 12500 	 Batches: 98
Data at bob: 12500 	 Batches: 98
Data at mike: 12500 	 Batches: 98
Data at rose: 12500 	 Batches: 98


In [39]:
# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
# net = RegNetX_200MF()
client_net = ResNet18()
client_net = client_net.to(device)
alice_net = copy.deepcopy(client_net)
bob_net = copy.deepcopy(client_net)
mike_net = copy.deepcopy(client_net)
rose_net = copy.deepcopy(client_net)
server_net = nn.Linear(512, num_classes)
server_net = server_net.to(device)
        

==> Building model..


In [32]:
nets = [alice_net, bob_net, mike_net, rose_net]

In [33]:
optimizers = [optim.SGD(net.parameters(), lr=args.lr,
                      momentum=0.9, weight_decay=5e-4) for net in nets]

In [34]:
server_optimizer = optim.SGD(server_net.parameters(), lr=args.lr,
                      momentum=0.9, weight_decay=5e-4)

In [41]:
server_loss = nn.CrossEntropyLoss()

In [42]:
epochs = 20
for e in range(epochs):
  running_loss = 0
  for i in range(len(data_loaders[0])):
    for opt in optimizers:
      opt.zero_grad()
    
    server_optimizer.zero_grad()

    lst_of_vars = []

    for i in range(len(data_loaders)): 
      imgs, lbls = next(iter(data_loaders[i]))
      imgs, lbls = imgs.to(device), lbls.to(device)
      split_output = nets[i](imgs.float()) 
      split_layer_output = split_output.clone().detach().requires_grad_(True)
      server_output = server_net(split_layer_output)
      
      loss = server_loss(server_output, lbls)

      lst_of_vars.append({'split_output': split_output, 'split_layer_output': split_layer_output, 'loss':loss})

    loss = 0  
    for i in range(len(data_loaders)):
      loss += lst_of_vars[i]['loss']

    avg_loss = loss / len(data_loaders)
    running_loss += avg_loss

    avg_loss.backward()

    for i in range(len(data_loaders)):
      split_gradients = lst_of_vars[i]['split_layer_output'].grad.clone().detach()
      lst_of_vars[i]['split_output'].backward(split_gradients)
    
    server_optimizer.step()
    
    for opt in optimizers:
      opt.step()

  print("Epoch {} - Training loss: {}".format(e+1, running_loss/len(data_loaders[0])))


Epoch 1 - Training loss: 1.7476544380187988
Epoch 2 - Training loss: 1.4216715097427368
Epoch 3 - Training loss: 1.2468806505203247
Epoch 4 - Training loss: 1.1071350574493408
Epoch 5 - Training loss: 1.0013169050216675
Epoch 6 - Training loss: 0.8944705128669739
Epoch 7 - Training loss: 0.8200480341911316
Epoch 8 - Training loss: 0.7573748826980591
Epoch 9 - Training loss: 0.6992132067680359
Epoch 10 - Training loss: 0.6631214022636414


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data 
        outputs = bob_net(images)
        #outputs = test_transform(outputs)
        outputs = server_net(outputs.float())

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the 10000 test images using SPLIT INFERENCE: %d %%' % (100 * correct / total))